In [6]:
import time
import gspread
import requests

from oauth2client.service_account import ServiceAccountCredentials

# Define exchange URLs
exchange_urls = {
    "Wazirx": "https://api.wazirx.com/api/v2/tickers",
    "CoinDCX": "https://api.coindcx.com/exchange/ticker",
    "Coinswitch": "https://cs-india.coinswitch.co/api/v2/external/csk_website/currencies",
    "Mudrex": "https://mudrex.com/api/v1/web/coins?sort=price&filter=price&order=desc&min=0&max=9007199254740991&size=500&offset=0",
}

# Define column names
column_names_wazirx = ["Base Unit", "Quote Unit", "Sell", "Buy", "Name"]

# Google Sheets credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/pawan/Downloads/creds.json', scope)
gc = gspread.authorize(credentials)

# Open the Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/1k9vZJsj8vSzxu7r7rQyrcBj-2Cnw5lTNPVgbY59Sr3I/edit?pli=1#gid=0'
sheet = gc.open_by_url(sheet_url).sheet1

def fetch_exchange_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data: {e}")
        return None

def update_sheet(data, header=None):
    if header:
        sheet.append_row(header)

    for item in data:
        sheet.append_row(item)

def update_wazirx_sheet():
    data = fetch_exchange_data(exchange_urls.get("Wazirx"))
    if data:
        
        sorted_data = sorted(data.items(), key=lambda x: float(x[1].get('buy', 0)), reverse=True)[:20]
        parsed_data = [[value.get('base_unit', ''), value.get('quote_unit', ''),
                        value.get('sell', ''), value.get('buy', ''),
                        value.get('name', '')] for _, value in sorted_data]
        update_sheet([column_names_wazirx])
        update_sheet(parsed_data, ["Wazirx"])
        

def update_coindcx_sheet():
    data = fetch_exchange_data(exchange_urls.get("CoinDCX"))
    if data:
        parsed_data = []
        sorted_data = sorted(data, key=lambda x: float(x.get('ask', 0)), reverse=True)[:20]
        for market_data in sorted_data:
            if market_data.get('market', '') != "BTCINR_insta":
                parsed_data.append([market_data.get('market', ''), market_data.get('change_24_hour', ''),
                                     market_data.get('last_price', '')])

        update_sheet(parsed_data, ["CoinDCX"])

def update_coinswitch_sheet():
    data = fetch_exchange_data(exchange_urls.get("Coinswitch"))
    if data and 'data' in data:
        currencies = data['data']['currencies']
        sorted_currencies = sorted(currencies, key=lambda x: float(x.get('price', '').replace(',', '')), reverse=True)[:20]
        parsed_data = [[currency.get('currency', ''), currency.get('currency_name', ''),
                        currency.get('price', '').replace(',', '')] for currency in sorted_currencies]

        update_sheet(parsed_data, ["Coinswitch"])

def update_mudrex_sheet():
    data = fetch_exchange_data(exchange_urls.get("Mudrex"))
    if data and 'data' in data:
        coins_data = data['data']
        sorted_coins = sorted(coins_data, key=lambda x: float(x.get('price_info', {}).get('price', 0)), reverse=True)[:20]
        parsed_data = [[coin.get('token', ''), coin.get('name', ''),
                        coin.get('price_info', {}).get('price', '')] for coin in sorted_coins]

        update_sheet(parsed_data, ["Mudrex"])

def main():
    try:
        sheet.clear()  
        while True:
            update_wazirx_sheet()
            time.sleep(10) 

            update_coindcx_sheet()
            time.sleep(10)  

            update_coinswitch_sheet()
            time.sleep(10) 

            update_mudrex_sheet()
            time.sleep(30)  

    except KeyboardInterrupt:
        print("Process interrupted. Exiting...")


if __name__ == "__main__":
    main()


Process interrupted. Exiting...
